In [1]:
import chess
import random
import time
from IPython.display import display, HTML, clear_output

board = chess.Board()

class Player:
    def next_move(self, board: chess.Board):
        raise NotImplemented

class RandomPlayer(Player):
    def next_move(self, board: chess.Board):
        move = random.choice(list(board.legal_moves))
        return move.uci()

class HumanPlayer(Player):
    def next_move(self, board: chess.Board):
        moves = list(m.uci() for m in board.legal_moves)
        move = input(f'Allowed moves: {moves}')
        while move not in moves:
            move = input(f'{move} is an illegal move! Allowed moves: {moves}')
        return move

def who(player):
    return "White" if player == chess.WHITE else "Black"

def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

def play_game(player1, player2, visual="svg", pause=0.1):
    """
    playerN1, player2: functions that takes board, return uci move
    visual: "simple" | "svg" | None
    """
    use_svg = (visual == "svg")
    board = chess.Board()
    try:
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                uci = player1.next_move(board)
            else:
                uci = player2.next_move(board)
            name = who(board.turn)
            board.push_uci(uci)
            board_stop = display_board(board, use_svg)
            html = f"<b>Move {len(board.move_stack)} {name}, Play '{uci}':</b><br/>{board_stop}"
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (None, msg, board)
    result = None
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    if visual is not None:
        print(msg)
    return (result, msg, board)


In [ ]:
player1 = RandomPlayer()
player2 = HumanPlayer()
play_game(player1, player2)